In [1]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from requests_html import HTMLSession

import time

In [2]:
driver = webdriver.Chrome(executable_path='C:/Users/17178/Downloads/chromedriver_win32/chromedriver.exe')
driver.get('https://www.autonation.com/cars-for-sale?bd=suv&pagesize=72&cnd=used')
content = driver.page_source
soup = BeautifulSoup(content)

In [3]:
time.sleep(5)

#have to re-load for whatever reason, else manufacturer dropdown doesn't expand
driver.get('https://www.autonation.com/cars-for-sale?bd=suv&pagesize=72&cnd=used')

#accept cookies (uncovers load more bar)
cookies = driver.find_element_by_xpath('/html/body/div[4]/div[3]/button')
cookies.click()

time.sleep(5)

#read in data by manufacturer
for i in range(1,41):
    driver.get('https://www.autonation.com/cars-for-sale?bd=suv&pagesize=72&cnd=used')

    #show manufacturer options
    show_manufacturers = driver.find_element_by_xpath("/html/body/section/div[4]/an-root/ansrp-dtansearchresultspagelib/srp-root/div/div/an-search/div[1]/div[3]/div[1]/div[3]/an-srp-refiners/div[1]/an-panel[3]/div")
    show_manufacturers.click()
    
    time.sleep(5)
        
    manufacturer_filter = driver.find_element_by_xpath(f"/html/body/section/div[4]/an-root/ansrp-dtansearchresultspagelib/srp-root/div/div/an-search/div[1]/div[3]/div[1]/div[3]/an-srp-refiners/div[1]/an-panel[3]/div/div[2]/div/an-body/an-srp-refiner-make/div/div[{i}]")
    manufacturer_filter.click()

    time.sleep(5)

    while True:
        try:
            showmore = driver.find_element_by_xpath("/html/body/section/div[4]/an-root/ansrp-dtansearchresultspagelib/srp-root/div/div/an-search/div[1]/div[3]/div[3]/div[2]/div[1]/button")
            showmore.click()
            time.sleep(5)
        except NoSuchElementException:
            break  

    content = driver.page_source
    soup = BeautifulSoup(content)

    results_model_trim = []
    results_price = []
    results_source = []

    counter = 0

    for element in soup.findAll('div', {'class': 'col-12 px-0 resultsSRP'}):
        for model_trim in element.findAll('div', {'class': 'vehicle-stylename'}):
            results_model_trim.append(model_trim.contents[0])
            counter += 1
        for price in element.findAll('div', {'class': 'price-Value ml-auto ng-star-inserted'}):
            results_price.append(price.contents[0])
            results_source.append('autonation')

    results_year_make = []
    results_state = []
    results_miles  = []

    for x in range(0,counter):
        for year_make in element.findAll('a', {'id': f'srp-tile-vehiclename-{x}'}):
            results_year_make.append(year_make.contents[0])
        for state in soup.findAll('span', {'id': f'srp-tile-distance-{x}'}):
            results_state.append(state.contents[0][1:3])
        if element.findAll('span', {'id': f'srp-tile-mileage-{x}'}):
            miles = element.find('span', {'id': f'srp-tile-mileage-{x}'})
            results_miles.append(miles.contents[0])
        else:
            results_miles.append('missing')

    df_temp = pd.DataFrame(
            {'year_make': results_year_make,
             'model_trim': results_model_trim,
             'price': results_price,
             'state': results_state,
             'miles': results_miles,
             'source': results_source
            })

    if i == 1:
        df = df_temp
    else:
        df = pd.merge(df, df_temp, how='outer')
        del df_temp

df

,year_make,model_trim,price,state,miles,source
0,2019 Acura RDX,AWD w/Advance Pkg,"$32,995",MD,63K miles,autonation
1,2017 Acura MDX,SH-AWD w/Advance Pkg,"$34,301",MD,37K miles,autonation
2,2022 Acura MDX,SH-AWD w/A-Spec Package,"$55,990",VA,3K miles,autonation
3,2019 Acura RDX,AWD,"$28,970",NY,44K miles,autonation
4,2019 Acura RDX,AWD w/Technology Pkg,"$32,495",TN,64K miles,autonation
...,...,...,...,...,...,...
8597,2016 Volvo XC90,AWD 4dr T6 Momentum,"$31,595",CA,53K miles,autonation
8598,2019 Volvo XC60,T5 AWD R-Design,"$39,498",CA,38K miles,autonation
8599,2020 Volvo XC90,T5 AWD Momentum 7 Passenger,"$48,395",CA,17K miles,autonation
8600,2020 Volvo XC90,T6 AWD Inscription 7 Passenger,"$59,987",CA,10K miles,autonation


In [5]:
#convert miles to numeric value
df['miles'] = df['miles'].str.extract('([0-9]+)').astype(float)*1000

#convert price to numeric value
df['price'] = df['price'].replace({'\$':''}, regex=True)
df['price'] = df['price'].replace({'\*':''}, regex=True)
df['price'] = df['price'].replace({'\,':''}, regex=True)
df['price'] = df['price'].astype(int)

df

,year_make,model_trim,price,state,miles,source
0,2019 Acura RDX,AWD w/Advance Pkg,32995,MD,63000.0,autonation
1,2017 Acura MDX,SH-AWD w/Advance Pkg,34301,MD,37000.0,autonation
2,2022 Acura MDX,SH-AWD w/A-Spec Package,55990,VA,3000.0,autonation
3,2019 Acura RDX,AWD,28970,NY,44000.0,autonation
4,2019 Acura RDX,AWD w/Technology Pkg,32495,TN,64000.0,autonation
...,...,...,...,...,...,...
8597,2016 Volvo XC90,AWD 4dr T6 Momentum,31595,CA,53000.0,autonation
8598,2019 Volvo XC60,T5 AWD R-Design,39498,CA,38000.0,autonation
8599,2020 Volvo XC90,T5 AWD Momentum 7 Passenger,48395,CA,17000.0,autonation
8600,2020 Volvo XC90,T6 AWD Inscription 7 Passenger,59987,CA,10000.0,autonation


In [6]:
df.to_csv('C:/Users/17178/Documents/DS Projects/Nitorum Case/Data/autonation_suv.csv')